In [3]:
#%pip install --extra-index-url https://test.pypi.org/simple ../../EnergyPlus-OOEP/
#%pip install git+https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-Datasets.git
#%pip install ipywidgets tqdm gymnasium torch ray[rllib]

# 🤖 Reinforcement Learning

TODO description

Kickstart the simulation.

In [4]:
#import logging as _logging_
#_logging_.basicConfig(level='INFO')
#from energyplus.ooep.addons.logging import MessageLogger

from energyplus.ooep.addons.logging import ProgressLogger

from energyplus.ooep import (
    World,
    Model,
    Weather,
    Report,
)

from energyplus.dataset.basic import dataset as _epds_

simulator = World(
    input=World.InputSpecs(
        model=(
            _epds_.models / 'ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
        ),
        weather=(_epds_.weathers / 'USA_CO_Denver-Aurora-Buckley.AFB.724695_TMY3.epw'),
    ),
    output=World.OutputSpecs(
        #report=('/tmp/ooep-report-9e1287d2-8e75-4cf5-bbc5-f76580b56a69'),
    ),
    options=World.RuntimeOptions(
        recurring=True,
        #design_day=True,
    ),
).add(
    ProgressLogger(),
    #MessageLogger(),
)

  0%|          | 0/100 [00:00<?, ?it/s]

Configure the controller (an RLlib algorithm).

In [5]:
import numpy as _numpy_
import gymnasium as _gymnasium_

from energyplus.ooep.addons.rl import (
    VariableBox,
    SimulatorEnv,
)

from energyplus.ooep import (
    Actuator,
    OutputVariable,
)

from ray.rllib.algorithms.ppo import PPOConfig



config = (
    PPOConfig.from_dict({        
        'create_env_on_local_worker': True
    })
    .resources(num_gpus=1)
    .environment(
        SimulatorEnv, 
        env_config=SimulatorEnv.Config(
            action_space=_gymnasium_.spaces.Dict({
                'thermostat': VariableBox(
                    low=15., high=20.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='Zone Temperature Control',
                    control_type='Heating Setpoint',
                    key='CORE_MID',
                ))
            }),    
            observation_space=_gymnasium_.spaces.Dict({
                'temperature': VariableBox(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='People Air Temperature',
                    key='CORE_MID',
                )),
            }),
            reward_function=lambda __obs: 1,
            event_refs=[
                'begin_zone_timestep_after_init_heat_balance',
            ],
            simulator_factory=lambda simulator=simulator: simulator,
        )
    )
    .rollouts(
        #num_rollout_workers=10,
        num_rollout_workers=0,
        enable_connectors=False,
    )
    .framework("torch")
    .training(
        model={"fcnet_hiddens": [64, 64]},
        lr=0.0001,
    )
    .evaluation(
        #evaluation_interval=1,
        #evaluation_num_workers=0
    )
)


algo = config.build(use_copy=True)

/home/user@AD/lab/EnergyPlus-OOEP/.venv/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:500: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/user@AD/lab/EnergyPlus-OOEP/.venv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
2024-05-25 00:58:57,523	WARNING unified.py:56 -- Could not instantiate JsonLogger: cannot pickle '_hashlib.HMAC' objec

Train the algorithm.

In [6]:
simulator.awaitable.run()

<Task pending name='Task-5' coro=<Engine.run() running at /home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/awaitables.py:68>>

Exception ignored on calling ctypes callback function: <function EventManager._core_callback_setters.<locals>.<dictcomp>.<lambda>.<locals>.<lambda> at 0x7ff8d74fd9e0>
Traceback (most recent call last):
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 89, in <lambda>
    state, lambda _: trigger(StateEvent(spec=spec))
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 67, in trigger
    self.trigger(*args, **kwargs)
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 142, in trigger
    raise e
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 139, in trigger
    try: super().trigger(event, *args, **kwargs)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/events.py", line 43, in trigger
  

In [7]:
# TODO rm: force simulator to rerun to make the newly requested variables available
#simulator.stop()
# start training
for _ in range(5):
    print(algo.train())

2024-05-25 00:59:09,932	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 0.20000000000000004, 'cur_lr': 0.00010000000000000003, 'total_loss': 9.937246103184197, 'policy_loss': -0.0637053886047935, 'vf_loss': 10.0, 'vf_explained_var': 0.7400906451286808, 'kl': 0.004757339720402743, 'entropy': 1.4149406410032703, 'entropy_coeff': 0.0}, 'model': {}, 'num_grad_updates_lifetime': 465.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}, 'sampler_results': {'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_epi

Evaluate the algorithm.

In [8]:
algo.evaluate()

{'evaluation': {'sampler_results': {'episode_reward_max': 9023.0,
   'episode_reward_min': 9023.0,
   'episode_reward_mean': 9023.0,
   'episode_len_mean': 9024.0,
   'episode_media': {},
   'episodes_this_iter': 4,
   'policy_reward_min': {},
   'policy_reward_max': {},
   'policy_reward_mean': {},
   'custom_metrics': {},
   'hist_stats': {'episode_reward': [9023.0, 9023.0, 9023.0, 9023.0],
    'episode_lengths': [9024, 9024, 9024, 9024]},
   'sampler_perf': {'mean_raw_obs_processing_ms': 0.11319857229959117,
    'mean_inference_ms': 0.9809485921214974,
    'mean_action_processing_ms': 0.08844645103167123,
    'mean_env_wait_ms': 1.8602753165506953,
    'mean_env_render_ms': 0.0},
   'num_faulty_episodes': 0,
   'connector_metrics': {}},
  'episode_reward_max': 9023.0,
  'episode_reward_min': 9023.0,
  'episode_reward_mean': 9023.0,
  'episode_len_mean': 9024.0,
  'episode_media': {},
  'episodes_this_iter': 4,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_me

Stop the simulation environment when we are done!

In [9]:
simulator.stop()